In [4]:
%run ../../Utils/yp_utils.py

# Initial setup

In [5]:
paper_pmid = 11452010
paper_name = 'ni_snyder_2001' 

In [6]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [7]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/phenotypes.txt', sep='\t', header=None)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 127 x 2


In [10]:
original_data.head()

,0,1
0,bud8,unipolar
1,bud9,unipolar
2,ste20,unipolar
3,ist3,unipolar
4,bud13,unipolar


In [11]:
original_data['gene'] = original_data[0].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, gene, orf]
Index: []


In [15]:
original_data['data'] = 1

In [16]:
original_data = pd.pivot_table(original_data, index='orf', values='data', columns=1)

In [17]:
original_data[original_data.isnull()] = 0

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(127, 3)

# Load & process tested strains

In [20]:
files = ['homo1.xls','homo2.xls','homo3.xls','homo4.xls']

In [21]:
tested_list = []
for f in files:
    tested = pd.read_excel('raw_data/'+f, sheet_name='Sheet1', header=None)
    tested = tested.iloc[:,1]
    tested.columns = ['ORF']
    tested_list.append(tested)

In [22]:
tested = pd.concat(tested_list, axis=0).to_frame()

In [23]:
tested.head()

,1
0,ORF name
1,YAL068C
2,YAL067C
3,YAL066W
4,YAL065C


In [24]:
tested['orf'] = tested[1].astype(str)

In [25]:
tested['orf'] = clean_orf(tested['orf'])

In [26]:
typos = {'TAL004W':'YAL004W','YELOO1C':'YEL001C','KL187C':'YKL187C'}
tested['orf'] = tested['orf'].apply(lambda x: typos[x] if x in typos.keys() else x)

In [27]:
tested['orf'] = translate_sc(tested['orf'].values, to='orf')

In [28]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             1      orf
0     ORF name  ORFNAME
1397    YMR41W   YMR41W
0          NaN      NAN
1          NaN      NAN
2          NaN      NAN
...        ...      ...
662        NaN      NAN
663        NaN      NAN
664        NaN      NAN
759        NaN      NAN
0     ORF name  ORFNAME

[85 rows x 2 columns]


In [29]:
tested = tested.loc[t,:]

In [30]:
tested_orfs = tested['orf'].unique()

In [31]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YGL174W']

In [32]:
tested_orfs = list(tested_orfs) + missing

In [33]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [34]:
original_data.head()

1,axial,random,unipolar
orf,,,
YAL068C,0.0,0.0,0.0
YAL067C,0.0,0.0,0.0
YAL066W,0.0,0.0,0.0
YAL065C,0.0,0.0,0.0
YAL062W,0.0,0.0,0.0


# Prepare the final dataset

In [35]:
data = original_data.copy()

In [36]:
dataset_ids = [4921, 4922, 467]
datasets = datasets.reindex(index=dataset_ids)

In [37]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [38]:
data.head()

dataset_id,4921,4922,467
data_type,value,value,value
orf,,,
YAL068C,0.0,0.0,0.0
YAL067C,0.0,0.0,0.0
YAL066W,0.0,0.0,0.0
YAL065C,0.0,0.0,0.0
YAL062W,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [39]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 25


In [40]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4921,4922,467
,data_type,value,value,value
gene_id,orf,,,
1869,YAL068C,0.0,0.0,0.0
61,YAL067C,0.0,0.0,0.0
60,YAL066W,0.0,0.0,0.0
1727,YAL065C,0.0,0.0,0.0
57,YAL062W,0.0,0.0,0.0


# Normalize

In [41]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [42]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [43]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4921,4922,467,4921,4922,467
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
1869,YAL068C,0.0,0.0,0.0,0.0,0.0,0.0
61,YAL067C,0.0,0.0,0.0,0.0,0.0,0.0
60,YAL066W,0.0,0.0,0.0,0.0,0.0,0.0
1727,YAL065C,0.0,0.0,0.0,0.0,0.0,0.0
57,YAL062W,0.0,0.0,0.0,0.0,0.0,0.0


# Print out

In [44]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [46]:
# from IO.save_data_to_db3 import *

In [47]:
# save_data_to_db(data_all, paper_pmid)